# Разборное разбираем

In [1]:
import string
import pandas as pd
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
from razdel import tokenize
from nltk.tokenize import word_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kirillkonca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
a = SovietRomaniAnalyzer(mode='strict')

# Borrowings Analyses

In [6]:
morph = MorphAnalyzer()

In [1]:
pym2uni = {'masc': 'm',
           'femn':'f',
           'neut': 'n',
            None: '',
           'sing': 'sg',
           'plur': 'pl',
           'NOUN': 'N'}

In [2]:
suffs = [(['obl', 'm', 'sg'], 'с'),
         (['gen', 'm', 'sg'], 'скирэ'),
         (['gen', 'm', 'sg'], 'скир'),
         (['gen', 'm', 'sg'], 'скр'),
         (['gen', 'f', 'sg'], 'кир'),
         (['gen', 'f', 'sg'], 'кр'),
         (['dat', 'm', 'sg'], 'скэ'),
         (['dat', 'f', 'sg'], 'кэ'),
         (['ins', '', 'sg'], 'са'),
         (['loc', 'm', 'sg'], 'стэ'),
         (['loc', 'f', 'sg'], 'тэ'),
         (['abl', 'm', 'sg'], 'стыр'),
         (['abl', 'f', 'sg'], 'тыр'),
         (['obl', '', 'pl'], 'н'),
         (['gen', '', 'pl'], 'нгир'),
         (['gen', '', 'pl'], 'нгр'),
         (['abl', '', 'pl'], 'ндыр'),
         (['dat', '', 'pl'], 'нгэ'),
         (['loc', '', 'pl'], 'ндэ'),
         (['ins', '', 'pl'], 'нса'),
         (['ins', '', 'pl'], 'нца'),
         (['voc', '', 'pl'], 'лэ')]

In [3]:
def stem2rom(stem, gramm, word, lemma=''):
    stem = stem.replace('жэ', 'же')  # жертва
    stem = stem.replace('жы', 'жи')
    stem = stem.replace('шы', 'ши')

    ru_analys = morph.parse(stem)[0]

    notfakedict = str(ru_analys.methods_stack[0][0]) != 'FakeDictionary()'
    whole_an = len(ru_analys.methods_stack[0][1]) == len(stem)
    name = word.istitle()
    good_score = ru_analys.score >= 0.2
    
    if name or (notfakedict and whole_an and good_score):
        if ru_analys.tag.POS == 'NOUN':
            trans_ru = ru_analys.normal_form
            #trans_en = translator.translate(trans_ru, src='ru').text.lower()
            if lemma == '':
                lemma = trans_ru
            if gramm[1] == '':
                gender = pym2uni[ru_analys.tag.gender]
            else:
                gender = gramm[1]
            if gramm[2] == '':
                number = pym2uni[ru_analys.tag.number]
            else:
                number = gramm[2]
            wfGlossed = word.lower()
            #gloss = trans_en
            if gramm[0] != '':
                wfGlossed = stem + '-' + word[len(stem):]
                #gloss = trans_en + '-' + (gramm[0]).upper() 
            rom_analys = {'wf': word,
                          'lemma': lemma,
                          'gramm': ['N',
                                    gender,
                                    gramm[0],
                                    number],
                          'wfGlossed': wfGlossed,
                          'trans_ru': trans_ru}
            return rom_analys

In [4]:
def borrowing_an(word):
    unknown = {'wf': word,
               'lemma': '',
               'gramm': [],
               'wfGlossed': '',
               'gloss': '',
               'trans_en': '',
               'trans_ru': ''}
    word_nojo = word.replace('ё', 'е')

    for gramm, suff in suffs:
        if word.endswith(suff):
            stems = [word[:-len(suff)], word[:-len(suff)-1],
                     word_nojo[:-len(suff)], word_nojo[:-len(suff)-1]]
            if word[:-len(suff)].endswith('ь'):
                stems.append(word[:-len(suff)-2])
            for stem in stems:
                rom_analys = stem2rom(stem, gramm, word)
                if rom_analys != None:
                    return rom_analys
    
    if word.endswith('ё') or word.endswith('о') or word.endswith('э'):
        rom_analys = stem2rom(word[:-1], ['', '', 'dir'], word, lemma=word) #вокатив, дир
        if rom_analys != None:
            return rom_analys

    rom_analys = stem2rom(word, ['', '', ''], word)
    if rom_analys != None:
        return rom_analys
    
    return unknown

# Corpus Annotation

Read file with pairs. Every text has to be annotated separately

In [693]:
import pandas as pd
df = pd.read_csv("pairs/zlodei.csv", sep=";")

Delete not full quatations

In [694]:
for i in range(len(df)):
    if str(df['to'][i]) != "nan":
        if (df['to'][i][-1] == "»" or df['to'][i][-1] == "».") and df['to'][i][0] != "«":
            df['to'][i] = df['to'][i][:-1]
        if df['to'][i][0] == "«" and df['to'][i][-1] != "»":
            df['to'][i] = df['to'][i][0:]
        if df['to'][i][0] == '"' and df['to'][i][-1] != '"':
            df['to'][i] = df['to'][i][0:]
        if df['to'][i][-1] == '"' and df['to'][i][0] != '"':
            df['to'][i] = df['to'][i][:-1]
        if df['to'][i][0] == '(' and df['to'][i][-1] != ')':
            df['to'][i] = df['to'][i][0:]
        if df['to'][i][-1] == ')' and df['to'][i][0] != '(':
            df['to'][i] = df['to'][i][:-1]
    if str(df['from'][i]) != "nan":
        if df['from'][i][-1] == "»" and df['from'][i][0] != "«":
            df['from'][i] = df['from'][i][:-1]
        if df['from'][i][0] == "«" and df['from'][i][-1] != "»":
            df['from'][i] = df['from'][i][0:]
        if df['from'][i][0] == '"' and df['from'][i][-1] != '"':
            df['from'][i] = df['from'][i][0:]
        if df['from'][i][-1] == '"' and df['from'][i][0] != '"':
            df['from'][i] = df['from'][i][:-1]
        if df['from'][i][0] == '(' and df['from'][i][-1] != ')':
            df['from'][i] = df['from'][i][0:]
        if df['from'][i][-1] == ')' and df['from'][i][0] != '(':
            df['from'][i] = df['from'][i][:-1]

/var/folders/n6/_05m2dl943qgt_yt01lb8h_w0000gn/T/ipykernel_50305/1143294128.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['to'][i] = df['to'][i][:-1]
/var/folders/n6/_05m2dl943qgt_yt01lb8h_w0000gn/T/ipykernel_50305/1143294128.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['to'][i] = df['to'][i][0:]


In [695]:
df = df.reset_index()

In [696]:
sentences_ru = df['to'].tolist()

In [697]:
sentences_roma = df['from'].tolist()

Delete byte tags

In [701]:
for i in range(len(sentences_roma)):
    if str(sentences_roma[i]) != "nan":
        sentences_roma[i] = sentences_roma[i].replace("\ufeff", "")
        sentences_roma[i] = sentences_roma[i].replace("\ufeff ", "")
        sentences_roma[i] = sentences_roma[i].replace("\xa0", "")
        sentences_roma[i] = sentences_roma[i].replace("\xa0 ", "")
        sentences_roma[i] = sentences_roma[i].replace("\xad", "")
        sentences_roma[i] = sentences_roma[i].replace("\xad ", "")
        sentences_roma[i] = sentences_roma[i].replace("\t", " ")

In [703]:
for i in range(len(sentences_ru)):
    if str(sentences_ru[i]) != "nan":
        sentences_ru[i] = sentences_ru[i].replace("\ufeff", "")
        sentences_ru[i] = sentences_ru[i].replace("\ufeff ", "")
        sentences_ru[i] = sentences_ru[i].replace("\xa0", "")
        sentences_ru[i] = sentences_ru[i].replace("\xa0 ", "")
        sentences_ru[i] = sentences_ru[i].replace("\xad", "")
        sentences_ru[i] = sentences_ru[i].replace("\xad ", "")
        sentences_ru[i] = sentences_ru[i].replace("-\t", " ")

Annotation

In [705]:
final_dict = {"corpus": []}

In [707]:
for i in tqdm(range(len(sentences_roma))):
    if str(sentences_roma[i]) != "nan" and str(sentences_roma[i]) != ' ':
        toks_in_sents = [word_tokenize(sent) for sent in sent_tokenize(sentences_roma[i])]
        analyses = a.analyze_words(toks_in_sents, format="json")
        analyses = analyses[0]
    else:
        analyses = []
    roma_words = []
    for an in analyses:
        words = []
        word = an[0]
        # Analyses each part of words with hyphens separately
        if len(word["wf"].split("-")) > 1:
            for w in word["wf"].split("-"):
                an_w = a.analyze_words(w, format="json")
                if len(an_w[0]["wf"]) == 0:
                    an_w = [borrowing_an(an_w[0]['wf'])]
                for pos_an in an_w:
                    words.append(pos_an)
            break
        # Analyse borrowing if not annotated with uniparser
        if len(word["lemma"]) == 0 and word["wf"] not in string.punctuation:
            an_w = borrowing_an(word["wf"])
            words.append(an_w)
        else:
            for al in an:
                words.append(al)
        roma_words.append(words)
    for mn in roma_words:
        for var in mn:
            if "trans_ru" not in var:
                var["trans_ru"] = ""
            if "trans_en" not in var:
                var["trans_en"] = ""
    ru = sent_tokenize(sentences_ru[i])
    ru = word_tokenize(ru[0])
    final_dict["corpus"].append({"sentence_ru": sentences_ru[i], "sentence_roma": sentences_roma[i], "sentence_id": i, "words_roma": roma_words})

100%|█████████████████████████████████████████| 676/676 [00:16<00:00, 40.37it/s]


In [311]:
with open('zlodei.json', 'w') as f:
    f.write(json.dumps(final_dict, indent=4, ensure_ascii=False))